### 5.1 スクレイピング

### 5.1.3 Webページをダウンロード

In [1]:
import requests


r = requests.get('https://www.seshop.com/')  # URLにアクセス
print(type(r))  # HTMLを表示
print(r.status_code)  # ステータスコードを表示

<class 'requests.models.Response'>
200


In [2]:
text = r.text   # HTMLのソースコードを取得する
for line in text.split('\n'):
    if '<title>' in line or '<h2>' in line:
        print(line.strip())

<title>SEshop｜ 翔泳社の本・電子書籍通販サイト</title>
<h2>書籍ランキング <span class="pull-right"><a href="/ranking/1"><span class="glyphicon glyphicon-chevron-right"></span> もっと見る</a></span></h2>
<h2>電子書籍ランキング<span class="pull-right"><a href="/ranking/327/"><span class="glyphicon glyphicon-chevron-right"></span> もっと見る</a></span></h2>
<h2>予約ランキング</h2>
<h2>新刊案内</h2>
<h2>SEshop限定　紙書籍と電子書籍のセット商品</h2>
<h2>申し込み受付中のセミナー</h2>
<h2>商品カテゴリー</h2>
<h2>人気商品ランキング</h2>
<h2>キャンペーン・特集</h2>


### 5.1.4 Webページから要素を抜き出す

In [3]:
from bs4 import BeautifulSoup

# HTMLを解析したBeautifulSoupオブジェクトを生成
soup = BeautifulSoup(text, 'html.parser')
print(soup.title)  # <title> タグの情報を取得
print(soup.h2)  # <h2> タグの情報を取得
# h2タグの中のaタグのhref属性
print(soup.h2.a['href'])

<title>SEshop｜ 翔泳社の本・電子書籍通販サイト</title>
<h2>書籍ランキング <span class="pull-right"><a href="/ranking/1"><span class="glyphicon glyphicon-chevron-right"></span> もっと見る</a></span></h2>
/ranking/1


In [4]:
atags = soup.find_all('a')  # すべてのaタグを取得
print(f"aタグの数: {len(atags)}")  # aタグの数を取得
for atag in atags[:5]:
    print(f"タイトル: {atag.text}")  # aタグのテキストを取得
    print(f"リンク: {atag['href']}")  # aタグのリンクを取得

aタグの数: 253
タイトル:  ヘルプ
リンク: /help
タイトル:  法人のお客様へ
リンク: /campaign/corp
タイトル:  新規会員登録
リンク: https://www.seshop.com/regist/
タイトル:  ログイン
リンク: #modalLogin
タイトル: 
リンク: /


### 5.1.5 書籍の一覧を抜き出す

In [5]:
from datetime import datetime


r = requests.get('https://www.seshop.com/product/616')
soup = BeautifulSoup(r.text, 'html.parser')

books = []  # 各書籍の情報を格納するリスト

# CSSセレクタで <div class="list">の中の<div class="inner">を取得
divs = soup.select("div.list div.inner")
for div in divs:
    img_url = div.find("img")["src"]
    # 日付の文字列を取得
    day = div.find("span", class_="date").text.strip()
    day = day.replace("発売", "")
    # 日付をdatetimeに変換
    published = datetime.strptime(day, "%Y.%m.%d")

    div_txt = div.find("div", class_="txt")
    a_tag = div_txt.find("a")  # aタグを取得
    title = a_tag.text.strip()  # 書籍タイトルを取得
    url = a_tag["href"]  # 書籍URLを取得

    # 販売価格を取得
    price_s = div_txt.find("span", class_="sale-price").parent.text
    price_s = price_s.strip()
    price_s = price_s.replace("円（税込）送料無料", "")
    price_s = price_s.replace(",", "")
    price = int(price_s)

    book = {
        "title": title,
        "img_url": img_url,
        "url": url,
        "price": price,
        "published": published,
    }
    books.append(book)

In [7]:
books[:3]

[{'title': '現場で使える！NumPyデータ処理入門 第2版  機械学習・データサイエンスで役立つ高速処理手法',
  'img_url': '/static/images/product/26383/L.png',
  'url': '/product/detail/26383',
  'price': 4400,
  'published': datetime.datetime(2024, 8, 26, 0, 0)},
 {'title': '動かして学ぶ！Python Django開発入門 第3版',
  'img_url': '/static/images/product/26360/L.png',
  'url': '/product/detail/26360',
  'price': 3960,
  'published': datetime.datetime(2024, 8, 9, 0, 0)},
 {'title': 'いきなりプログラミング Python',
  'img_url': '/static/images/product/26239/L.png',
  'url': '/product/detail/26239',
  'price': 2420,
  'published': datetime.datetime(2024, 6, 25, 0, 0)}]

In [8]:
import pandas as pd


df = pd.DataFrame(books)
df.head()

,title,img_url,url,price,published
0,現場で使える！NumPyデータ処理入門 第2版 機械学習・データサイエンスで役立つ高速処理手法,/static/images/product/26383/L.png,/product/detail/26383,4400,2024-08-26
1,動かして学ぶ！Python Django開発入門 第3版,/static/images/product/26360/L.png,/product/detail/26360,3960,2024-08-09
2,いきなりプログラミング Python,/static/images/product/26239/L.png,/product/detail/26239,2420,2024-06-25
3,爆速Python,/static/images/product/26287/L.png,/product/detail/26287,3960,2024-06-24
4,【SEshop限定】Python1-2年生 プログラミング初心者向け応援セット◆「ヤギ博士＆...,/static/images/product/26392/L.png,/product/detail/26392,7018,2024-06-20
